# A notebook example to create some fairly complex models

In [ ]:
!python -m pip uninstall -y bokeh holoviews synthetic_model sep_python


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
!python -m pip install "synthetic_model @ git+https://github.com/SEP-software/synthetic-model.git@9efd32607bc31a4db565a7440b830eb8883e8822"


## A function to plot frames from the model

In [ ]:
import numpy as np
from synthetic_model import Hypercube
def createPlot(data,p1,p2,p3):
    if isinstance(data,np.ndarray):
        fld=data
        hyper=hypercube(ns=[data.shape[2],data.shape[1],data.shape[0]])
    else:
        fld=data.get_nd_array()
        hyper=data.get_hyper()
    ax1=hyper.axes[0]
    ax2=hyper.axes[1]
    ax3=hyper.axes[2]
    mn=fld.min()
    mx=fld.max()
    front=fld[p3,:,::-1].T
    top=fld[::-1,:,p1]
    side=fld[:,p2,::-1].T
    b=(ax1.o,ax1.o+ax1.d*(ax1.n-1),ax2.o,ax2.o+ax2.d*(ax2.n-1))
    frontF=hv.Image(front,kdims=["X","Z"],bounds=(ax2.o,ax1.o,ax2.o+ax2.d*(ax2.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,invert_yaxis=True,aspect=1,axiswise=True)
    sideF=hv.Image(side,kdims=["Y","Z"],bounds=(ax3.o,ax1.o,ax3.o+ax3.d*(ax3.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,yaxis=None,invert_yaxis=True,aspect=1.,axiswise=True)
    topF=hv.Image(top,kdims=["X","Y"],bounds=(ax2.o,ax2.o,ax2.o+ax2.d*(ax2.n-1),ax3.o+ax3.d*(ax3.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,xaxis=None,aspect=1,axiswise=True)
    botF=hv.Layout(frontF+sideF).opts(toolbar=None)
    tot=hv.Layout(topF+hv.Empty()+frontF+sideF).cols(2).opts(toolbar=None)
    return tot
    

## A function to convert an image into a numpy array to be inserted into the model

In [ ]:
import numpy as np
from PIL import Image

def png_to_np_array(image_path):
    # Open the image using PIL
    img = Image.open(image_path)
    img = img.convert('RGB')

    # Compute new width while keeping the aspect ratio
    aspect_ratio = img.width / img.height
    new_width = int(60 * aspect_ratio)
    
    # Resize the image
    img_resized = img.resize((new_width, 60))
    
    # Convert resized image to numpy array
    np_img = np.asarray(img_resized)
    
    # Convert values to float ranging between 0 and 1
    np_img_float = np_img.astype(float) / 255.0
    
    return np_img_float

# Example usage:
image_path = "./logo.png"
res = png_to_np_array(image_path).transpose(1,0,2)
logo=np.zeros((104,60))
logo[:,:]= (res[:104,:,0]+res[:104,:,1]+res[:104,:,2])
logo=1-logo/3
logo=logo*4000

## The insert function

In [ ]:
from numba import njit
@njit
def add_logo(ar,logo,sc_3,c3,l3,b2,b1):
    b3=c3-l3-sc_3
    e3=c3+l3+sc_3-1
    for i3 in range(sc_3):
        sc=i3/sc_3
        for i2 in range(logo.shape[0]):
            for i1 in range(logo.shape[1]):
                if logo[i2,i1] >.01:
                    ar[b3+i3,b2+i2,b1+i1]=(1.-sc)*ar[b3+i3,b2+i2,b1+i1]+sc*logo[i2,i1]
                    ar[e3-i3,b2+i2,b1+i1]=(1.-sc)*ar[b3+i3,b2+i2,b1+i1]+sc*logo[i2,i1]
    
    for i2 in range(logo.shape[0]):
        for i1 in range(logo.shape[1]):
            if logo[i2,i1] >.01:
                ar[c3-l3:c3+l3,b2+i2,b1+i1]=logo[i2,i1]


## A function to set eps and delta to reasonable values

In [ ]:
from numba import njit
@njit
def cleanup_eps_delta(eps,delta,eps_clip,delta_clip_ratio):
    for i3 in range(eps.shape[2]):
        for i2 in range(eps.shape[1]):
            for i1 in range(eps.shape[0]):
                if eps[i3,i2,i1] < eps_clip or eps[i3,i2,i1]<=0.:
                    eps[i3,i2,i1]=0
                    delta[i3,i2,i1]=0.
                elif delta[i3,i2,i1]/eps[i3,i2,i1] > delta_clip_ratio:
                    delta[i3,i2,i1]=eps[i3,i2,i1]*delta_clip_ratio
                elif delta[i3,i2,i1]<0.:
                    delta[i3,i2,i1]=0.


## Basic setup



In [ ]:
import holoviews as hv
from synthetic_model import GeoModel,Deposit,Squish,ErodeFlat,Fault
hv.extension("bokeh","matplotlib",logo=False)

# Create a hypercube describing the basement
hyper=Hypercube.set_with_ns(ns=[20,2400,1000],os=[0,0,0],ds=[10,10,10])

#Create the initial geomodel
basement=GeoModel(hyper,{"vp":5000.,"vs":3000,"rho":2400,"eps":0,"delta":0},"vp")

#Create several different default deposition modes
depositC=Deposit()  #Constant velocity
depositI=Deposit(interbedThickAvg=20.,interbedThickDev=13.,interbedPropDev=100.) #Interbeded layer
depositN=Deposit(noiseDev=300.) #Add some noise to constant velocity\
depositIC=Deposit(interbedThickAvg=15.,interbedThickDev=13.,interbedPropDev=100.,interbedDev=4.) #Interbed with noise
depositICN=Deposit(noiseDev=120,  #Spatial variation of vp
                   interbedThickDev=18., #std-deviation of thickness
                   noiseFrequency=50, #How fast spatially velocity varies
                   noiseZStretch=10, #Make the noise vary more in Z than X,Y
                   interbedPropDev=240.,#How much the interbed velocity varies in VP on average
                    interbedDev=15, #How much thickness of the layer thickness change in Z
                   interbedFreq=4, #How quickly the thickness changes in X,Y
                   interbedThickAvg=30, #Average thickness of layers
                   vs_ratio=.65,  #Average vs ratio for the sequence
                   vs_dev=90,  #Spatial variation in vs
                   rho_dev=100, # Spatial variation in rho
                   eps_prop=.09, #Average epsilon change
                   eps_interbedPropDev=.06, #Average Chaange in eps z
                   eps_depthGrad=0., #Epsilon gradient
                   eps_noiseDev=0.03, #Epsilon change in XY
                   delta_prop=.04, #Average delta 
                   delta_interbedPropDev=.03, #Average delta change in Z
                   delta_depthGrad=0., #Delta gradient
                   delta_noiseDev=.02 #Delta change in XY
                  )

## Apply the first depostion layer

Notes
- Setting Vs and rho as ratios of VP with some lateral deviation
- epsilon, delta follow the same layer structure but are independently defined

In [ ]:
model=depositICN.apply(basement,thick=100,
                       prop=4200,seed=23,
                       vs_ratio=.65,
                       delta_prop=.08,
                       eps_prop=.17
                      )


## Plot vp at this stage

In [ ]:
plt=createPlot(model.getFloatField("vp"),60,1200,500)
plt


# Build more of the model


## Fault the model

#Loop through applying faults

In [ ]:
#Introduce lateral up-down pattern
squished=Squish().apply(model,begFreq=70,maxShift=210,widthInLine=300,
                        widthCrossLine=500,seed=234)
#Depsit another layer sequence
model=depositICN.apply(model,thick=60,prop=3500,
                       
                        vs_ratio=.72,
                       rho_ratio=2300/3500,
                       eps_prop=.12,
                       delta_prop=.08,
                           seed=334)
#Deposit another layer sequence
model=depositICN.apply(model,thick=80, 
                       prop=2800,
                        vs_ratio=.72,
                       rho_ratio=2300/2800,
                       eps_prop=.12,
                       delta_prop=.08,
                           seed=334)
#Introduce another up down pattern
model=Squish().apply(model,begFreq=70,maxShift=310,widthInLine=320,
                        widthCrossLine=500,seed=223)

#Deposit another layer
model=depositICN.apply(model,thick=3,prop=2100,
                        vs_ratio=.61,
                       rho_ratio=2300/2100,
                       eps_prop=.12,
                       delta_prop=.08,
                           seed=205)
#

In [ ]:
#Create parameters for a fault sequence
import random
import numpy as np
np.random.seed(22)
nsz=30
xs=np.random.uniform(low=.1,high=.9,size=(nsz))
ys=np.random.uniform(low=.1,high=.9,size=(nsz))
zs=np.random.uniform(low=.32,high=.8,size=(nsz))
dz=np.random.uniform(low=-.03,high=.03,size=(nsz))
angles=np.random.uniform(low=43,high=64,size=(nsz,2))
rads=np.random.uniform(low=3850,high=5200,size=(nsz))
shifts=np.random.uniform(low=300,high=750,size=(nsz))
lengths=np.random.uniform(low=2.8,high=3.4,size=(nsz))
lengths[:]*=shifts[:]
dl=np.random.uniform(low=-40,high=40,size=(nsz))
azimuth=np.random.uniform(low=-27,high=2,size=(nsz))
seeds=np.random.randint(99,size=(nsz))
dy=np.random.uniform(low=.12,high=.17,size=(nsz))
#Deposit a small layer
modelF=depositC.apply(model,thick=4,prop=1850,
                        vs_ratio=.65,vs_dev=0,
                       rho_ratio=2500/2100,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,   
                     )
x=modelF.get().get_nd_array()
print(x.min(),x.max())

In [ ]:
for i in range(nsz):
    print(f"{i} x={xs[i]} y={ys[i]} z={zs[i]}")
    #Fault
    modelF=Fault().apply(modelF,angle=angles[i,0],radius=rads[i],
                       begy=ys[i],begx=xs[i],\
            azimuth=azimuth[i],seed=seeds[i],indicateI=True,\
        begz=zs[i],shift=shifts[i],ruptureLength=lengths[i])
    #Add to the top of the model 
    modelF=depositC.apply(modelF,thick=1,prop=1850,
                    vs_ratio=.65,vs_dev=0,
                       rho_ratio=2200/2100,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,
                         )
    x=modelF.get().get_nd_array()
    print(x.min(),x.max(),x)

In [ ]:
x=modelF.get().get_nd_array()[543,1769,2]
y=modelF.getLayer().get_nd_array()[543,1769,2]
print(x.min(),x.max())
print(y)

## Finish up

In [ ]:
#Deposit a water layer
full=depositC.apply(modelF,prop=1450,
                        vs_ratio=.0,vs_dev=0,
                       rho_prop=1000,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,
                   )
ar=full.get().get_nd_array()
#Fix delta and epsion
cleanup_eps_delta(full.getFloatField("eps").get_nd_array(), 
                  full.getFloatField("delta").get_nd_array(),
                  .05,.8)

print(ar.min(),ar.max(),logo.min(),logo.max())
print(type(ar),type(logo))
#Add a logo
#add_logo(ar,logo,40,400,20,350,80)
#write out npz
full.writeNPZ("no_salt.npz")

## Plot the final model

In [ ]:
plt=createPlot(full.get(),250,1000,500)
plt

## Redo with some salt

In [ ]:
model=depositICN.apply(basement,thick=100, 
                       prop=4200,nterbedThickAvg=30,
                       vs_ratio=.71,
                       rho_ratio=2300/4200,
                       eps_prop=.15,
                       delta_prop=.08,
                           seed=334
                      )
model=depositICN.apply(model,thick=60,prop=3500,
                           vs_ratio=.72,
                       rho_ratio=2300/3500,
                       eps_prop=.12,
                       delta_prop=.08,
                           seed=3340)
model=depositICN.apply(model,thick=80,noiseDev=30, 
                       prop=2800,
                        vs_ratio=.6672,
                       rho_ratio=2300/3500,
                       eps_prop=.08,
                       delta_prop=.07,
                           seed=3344)
model=Squish().apply(model,begFreq=70,maxShift=210,widthInLine=100,
                        widthCrossLine=500,seed=223)
model=depositICN.apply(model,thick=3,prop=2100,
                        vs_ratio=.72,
                       rho_ratio=2300/2100,
                       eps_prop=.08,
                       delta_prop=.03,
                           seed=3534)
import random
import random
import numpy as np
np.random.seed(53)
nsz=50
xs=np.random.uniform(low=.2,high=.75,size=(nsz))
ys=np.random.uniform(low=.2,high=.75,size=(nsz))
zs=np.random.uniform(low=.35,high=.65,size=(nsz))
dz=np.random.uniform(low=-.03,high=.03,size=(nsz))
pdie=np.random.uniform(low=.05,high=.34,size=(nsz))

angles=np.random.uniform(low=43,high=64,size=(nsz,2))
rads=np.random.uniform(low=4850,high=5200,size=(nsz))
shifts=np.random.uniform(low=220,high=550,size=(nsz))
lengths=np.random.uniform(low=2.9,high=3.4,size=(nsz))
lengths[:]*=shifts[:]
dl=np.random.uniform(low=-40,high=40,size=(nsz))
azimuth=np.random.uniform(low=-20,high=4,size=(nsz))
seeds=np.random.randint(99,size=(nsz))
dy=np.random.uniform(low=.12,high=.17,size=(nsz))
modelF=depositC.apply(model,thick=4,prop=1850,
                        vs_ratio=.65,vs_dev=0,
                       rho_ratio=2500/2100,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,   
                     )
for i in range(nsz):
    print(azimuth[i],shifts[i],lengths[i])
    print(f"x={xs[i]} y={ys[i]} z={zs[i]}")
    modelF=Fault().apply(modelF,angle=angles[i,0],radius=rads[i],perp_die=pdie[i],
                       begy=ys[i],begx=xs[i],\
            azimuth=azimuth[i],seed=seeds[i],indicateI=True,\
        begz=zs[i],shift=shifts[i],ruptureLength=lengths[i])

    modelF=depositC.apply(modelF,thick=1,prop=1850,
                        vs_ratio=.65,vs_dev=0,
                       rho_ratio=2500/1850,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,   
                     )


#Add the salt layer

In [ ]:
from synthetic_model import Salt
salt=Salt().apply(modelF,applyHills=True,\
            salt_value={"vp":4500,"vs":2700,"rho":2300,"eps":0,"delta":0.
                       },thickness=.09,mid_salt_depth=.3,amplitude_top=.2,
                  beg2=.45,end2=.52,beg3=.45,end3=.55, seed=2,dieF=.1,
                  amplitude_bot=.2,beg_freq_top=15.,n_octaves_top=6,beg_freq_bot=7)

water=depositC.apply(salt,prop=1450,
                        vs_ratio=.0,vs_dev=0,
                       rho_prop=1000,rho_dev=0,
                       eps_prop=.0,eps_interbedPropDev=.0, 
                       eps_depthGrad=0.,eps_noiseDev=0.0,
                       delta_prop=.0,delta_interbedPropDev=.0, 
                       delta_depthGrad=0.,delta_noiseDev=.0,
                   )
ar=full.get().get_nd_array()
cleanup_eps_delta(water.getFloatField("eps").get_nd_array(), 
                  water.getFloatField("delta").get_nd_array(),
                  .05,.8)

In [ ]:
ar=water.get().get_nd_array()

#add_logo(ar,logo,40,400,20,350,240)
water.writeNPZ("salt.npz")

In [ ]:
plt=createPlot(water.get(),250,1000,500)
plt
